# The political and economic consequences of COVID-19

In [10]:
import numpy as np
import pandas as pd
import json
import OpenBlender
import config


In [11]:
# this code is taken from https://towardsdatascience.com/gather-all-the-coronavirus-data-with-python-19aa22167dea
# getting corona virus dataset from openblender
action = 'API_getObservationsFromDataset'
parameters = { 
 'token': config.token,
 'id_dataset':'5e6ac97595162921fda18076',
 'date_filter':{
               "start_date":"2020-01-01T06:00:00.000Z",
               "end_date":"2020-03-11T06:00:00.000Z"} ,
 
}

df_confirmed = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df_confirmed.reset_index(drop=True, inplace=True)
df_confirmed.head()


Task ID: '5e6d552f0895fafb4a9d8d4b'.
Total estimated consumption: 33672.88 processing units.
Task confirmed. Starting download..
6.25 %
12.5 %
18.75 %
25.0 %
31.25 %
37.5 %
43.75 %
50.0 %
56.25 %
62.5 %
68.75 %
75.0 %
81.25 %
87.5 %
93.75 %
100.0 % completed.


,confirmed,deaths,timestamp,provincestate,longitude,latitude,recovered,countryregion
0,120,3,1583884382,Hong Kong,114.2,22.3,65,Hong Kong SAR
1,491,3,1583884382,,8.2275,46.8182,3,Switzerland
2,1695,35,1583884382,,-4,40,32,Spain
3,576,6,1583884382,Chongqing,107.874,30.0572,547,Mainland China
4,133,0,1583884382,Shanxi,112.292,37.5777,131,Mainland China
5,400,0,1583884382,,8.4689,60.472,1,Norway
6,355,0,1583884382,,16,63,1,Sweden
7,382,6,1583884382,,-3,55,18,UK
8,92,0,1583878391,Massachusetts,-71.5301,42.2302,1,US
9,267,23,1583878391,Washington,-121.49,47.4009,1,US


In [23]:
# load the news stories which were extracted from Media Cloud
df_news = pd.read_csv('covid-19-or-coronavirus-or-all-story-urls-20200315082840.csv')
df_news.head()


,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
0,1483691901,2020-01-01 02:15:15,19 Things You Won't Understand If You Were Bor...,https://www.buzzfeed.com/daniellaemanuel/kids-...,en,False,NaN,6218,Buzzfeed,http://www.buzzfeed.com
1,1483736058,2020-01-01 00:00:28,"Kings ride power play, 4-goal first period pas...",https://www.dailynews.com/2019/12/31/kings-rid...,en,False,NaN,24901,bleacherreport.com,http://bleacherreport.com
2,1483742069,2020-01-01 00:22:23,"Neal records hat trick in 4-point game, Oilers...",https://www.kansascity.com/entertainment/artic...,en,False,NaN,30,The Kansas City Star,http://www.kansascity.com/
3,1483743956,2020-01-01 00:15:24,"Kane scores twice, leads Blackhawks to 5-3 win...",http://www.startribune.com/kane-scores-twice-l...,en,False,NaN,19,Star Tribune,http://www.startribune.com/
4,1483743953,2020-01-01 00:25:06,"Neal records hat trick in 4-point game, Oilers...",http://www.startribune.com/neal-records-hat-tr...,en,False,NaN,19,Star Tribune,http://www.startribune.com/


In [ ]:
# get financial data for commodities, stocks and currencies

